参照カウント方式のスマートポインタ型
  データに複数の所有者を持たせる
  所有者がいなくなったらデータの片付けをしてくれる

スマートポインタがDerefとDropトレイトを実装している

- ヒープに値を確保するBox<T>
- 複数の所有権を可能にする参照カウント型のRc<T>
- RefCell<T>を通してアクセスされ、コンパイル時ではなく実行時に借用規則を強制する型のRef<T>とRefMut<T>
- 内部可変性パターン
- 循環参照


# ヒープのデータを指すBox<T>を使用する

## Box<T>を使ってヒープにデータを格納する


In [3]:
{
  let b = Box::new(5);
  println!("b = {}", b);
}

b = 5


()

```
(32, (11, (10, (1, "Nil"))))
```

In [8]:
enum List {
  Cons(i32, List),
  Nil,
}

use List::{Cons, Nil};
let list = Cons(1, Cons(2, Cons(3, Nil)));

Error: recursive type `List` has infinite size

Error: cycle detected when computing drop-check constraints for `List`

```rust
enum Message {
    Quit,
    Move { x: i32, y: i32 },
    Write(String),
    ChangeColor(i32, i32, i32),
}
```


In [13]:

#[derive(Debug)]
enum List {
  Cons(i32, Box<List>),
  Nil,
}

use List::{Cons, Nil};
let list = Cons(1,
    Box::new(Cons(2,
      Box::new(Cons(3,
        Box::new(Nil))))));
list

Cons(1, Cons(2, Cons(3, Nil)))

In [4]:
{
  let x = 5;
  let y = &x;

  assert_eq!(5, x);
  assert_eq!(5, *y);
}

()

In [5]:
{
  let x = 5;
  let y = &x;

  assert_eq!(5, y);
}

Error: can't compare `{integer}` with `&{integer}`

In [6]:
{
  let x = 5;
  let y = Box::new(x);

  assert_eq!(5, x);
  assert_eq!(5, *y);
}

()

In [7]:
struct MyBox<T>(T);

impl<T> MyBox<T> {
  fn new(x: T) -> MyBox<T> {
    MyBox(x)
  }
}

In [8]:
{
  let x = 5;
  let y = MyBox::new(x);

  assert_eq!(5, x);
  assert_eq!(5, *y);
}

Error: type `MyBox<{integer}>` cannot be dereferenced

Derefトレイトを実装

In [9]:
use std::ops::Deref;

impl<T> Deref for MyBox<T> {
  type Target = T;

  fn deref(&self) -> &T {
    &self.0
  }
}

In [10]:
{
  let x = 5;
  let y = MyBox::new(x);

  assert_eq!(5, x);
  assert_eq!(5, *y);
}

()

In [16]:
struct DoubleBox<T>(T);

impl<T> DoubleBox<T> {
  fn new(x: T) -> DoubleBox<DoubleBox<T>> {
    DoubleBox(DoubleBox(x))
  }
}

use std::ops::Deref;

impl<T> Deref for DoubleBox<T> {
  type Target = T;

  fn deref(&self) -> &T {
    &self.0
  }
}

In [18]:
use std::ops::Deref;

impl<T> Deref for DoubleBox<T> {
  type Target = T;

  fn deref(&self) -> &T {
    &self.0
  }
}

In [19]:
{
  let x = 5;
  let y = DoubleBox::new(x);

  assert_eq!(5, x);
  assert_eq!(5, *y);
}

Error: can't compare `{integer}` with `DoubleBox<{integer}>`

Error: `DoubleBox<{integer}>` doesn't implement `Debug`

In [20]:
{
  let x = 5;
  let y = DoubleBox::new(x);

  assert_eq!(5, x);
  assert_eq!(5, **y);
}

()

参照外し型強制

In [11]:
fn hello(name: &str) {
  println!("Hello, {}!", name);
}

In [12]:
let m = MyBox::new(String::from("Rust"));
hello(&m);

Hello, Rust!


In [22]:
// call `hello` without implicit deref coercions
hello(&(*m)[..]);

Hello, Rust!


In [23]:
// number of times to `deref` insertion is resolved at compile tims.
let m = DoubleBox::new(String::from("Rust"));
hello(&m);

Hello, Rust!


## Dropトレイトで片付け時にコードを走らせる

In [3]:
struct CustomSmartPointer {
  data: String,
}

impl Drop for CustomSmartPointer {
  fn drop(&mut self) {
    println!("Dropping CustomSmartPointer with data `{}`!", self.data);
  }
}

In [5]:
{
  let c = CustomSmartPointer { data: String::from("my stuff") };
  let d = CustomSmartPointer { data: String::from("other stuff") };
  println!("CustomSmartPonters created.");
}

()

CustomSmartPonters created.
Dropping CustomSmartPointer with data `other stuff`!
Dropping CustomSmartPointer with data `my stuff`!


In [6]:
{
  let c = CustomSmartPointer { data: String::from("my stuff") };
  println!("CustomSmartPonters created.");
  c.drop();
  println!("CustomSmartPointer dropped before the end of main.");
}

Error: explicit use of destructor method

In [7]:
{
  let c = CustomSmartPointer { data: String::from("my stuff") };
  println!("CustomSmartPonters created.");
  drop(c);
  println!("CustomSmartPointer dropped before the end of main.");
}

CustomSmartPonters created.
Dropping CustomSmartPointer with data `my stuff`!
CustomSmartPointer dropped before the end of main.


()

## Rc<T>は、参照カウント方式のスマートポインタ

In [8]:
enum List {
  Cons(i32, Box<List>),
  Nil,
}

In [12]:
use List::{Cons, Nil};

let a = Cons(5,
  Box::new(Cons(10,
    Box::new(Nil))));

In [13]:
let b = Cons(3, Box::new(a));
let c = Cons(4, Box::new(a));

Error: use of moved value: `a`

In [21]:
use std::rc::Rc;

enum List {
  Cons(i32, Rc<List>),
  Nil,
}

In [22]:
use List::{Cons, Nil};

let a = Rc::new(Cons(5,
  Rc::new(Cons(10,
    Rc::new(Nil)))));

In [23]:
let b = Cons(3, Rc::clone(&a));
let c = Cons(4, Rc::clone(&a));

In [24]:
{
  let a = Rc::new(Cons(5, Rc::new(Cons(10, Rc::new(Nil)))));
  println!("count = {}", Rc::strong_count(&a));

  let b = Cons(3, Rc::clone(&a));
  println!("count = {}", Rc::strong_count(&a));

  {
    let c = Cons(4, Rc::clone(&a));
    println!("count = {}", Rc::strong_count(&a));
  }

  println!("count = {}", Rc::strong_count(&a));
}

count = 1
count = 2
count = 3
count = 2


()

## RefCell<T>と内部可変性パターン

借用規則
- いかなる時も(以下の両方ではなく、)1つの可変参照かいくつもの不変参照のどちらかが可能になる
- 参照は常に有効でなければならない。

In [2]:
let x = 5;
let y = &mut x;

Error: cannot borrow `x` as mutable, as it is not declared as mutable

Error: cannot use `x` because it was mutably borrowed

In [3]:
pub trait Messenger {
  fn send(&self, msg: &str);
}

In [5]:
pub struct LimitTracker<'a, T: 'a + Messenger> {
  messenger: &'a T,
  value: usize,
  max: usize,
}

In [7]:
impl<'a, T> LimitTracker<'a, T> where T: Messenger {
  pub fn new(messenger: &T, max: usize) -> LimitTracker<T> {
    LimitTracker {
      messenger,
      value: 0,
      max,
    }
  }

  pub fn set_value(&mut self, value: usize) {
    self.value = value;

    let percentage_of_max = self.value as f64 / self.max as f64;

    if percentage_of_max > 0.75 && percentage_of_max < 0.9 {
      self.messenger.send("Warning: You've used up over 75% of your quota!");
    } else if percentage_of_max > 0.9 && percentage_of_max < 1.0 {
      self.messenger.send("Urgent warning: You've used up over 90% of your quota!");
    } else if percentage_of_max >= 1.0 {
      self.messenger.send("Error: You are over your quota!");
    }
  }
}

In [14]:
// #[cfg(test)]
// mod tests {
//   use super::*;

  struct MockMessenger {
    sent_messages: Vec<String>,
  }

  impl MockMessenger {
    fn new() -> MockMessenger {
      MockMessenger { sent_messages: vec![] }
    }
  }

  impl Messenger for MockMessenger {
    fn send(&self, message: &str) {
      self.sent_messages.push(String::from(message));
    }
  }

  // #[test]
  // fn it_send_an_over_75_percent_waring_message() {
    let mock_messenger = MockMessenger::new();
    let mut limit_tracker = LimitTracker::new(&mock_messenger, 100);

    limit_tracker.set_value(80);
    assert_eq!(mock_messenger.sent_messages.len(), 1);
  // }
// }

Error: cannot borrow `self.sent_messages` as mutable, as it is behind a `&` reference

Error: `mock_messenger` does not live long enough

In [19]:
{
  use std::cell::RefCell;

  struct MockMessenger {
    sent_messages: RefCell<Vec<String>>,
  }

  impl MockMessenger {
    fn new() -> MockMessenger {
      MockMessenger { sent_messages: RefCell::new(vec![]) }
    }
  }

  impl Messenger for MockMessenger {
    fn send(&self, message: &str) {
      self.sent_messages.borrow_mut().push(String::from(message));
    }
  }

  // fn it_send_an_over_75_percent_waring_message() {
    let mock_messenger = MockMessenger::new();
    let mut limit_tracker = LimitTracker::new(&mock_messenger, 100);

    limit_tracker.set_value(80);
    assert_eq!(mock_messenger.sent_messages.borrow().len(), 1);
  // }
}

()

In [20]:
{
  use std::cell::RefCell;

  struct MockMessenger {
    sent_messages: RefCell<Vec<String>>,
  }

  impl MockMessenger {
    fn new() -> MockMessenger {
      MockMessenger { sent_messages: RefCell::new(vec![]) }
    }
  }

  impl Messenger for MockMessenger {
    fn send(&self, message: &str) {
      let mut one = self.sent_messages.borrow_mut();
      let mut two = self.sent_messages.borrow_mut();
      
      one.push(String::from(message));
      two.push(String::from(message));
    }
  }
}

()

In [21]:
{
  use std::cell::RefCell;

  struct MockMessenger {
    sent_messages: RefCell<Vec<String>>,
  }

  impl MockMessenger {
    fn new() -> MockMessenger {
      MockMessenger { sent_messages: RefCell::new(vec![]) }
    }
  }

  impl Messenger for MockMessenger {
    fn send(&self, message: &str) {
      let mut one = self.sent_messages.borrow_mut();
      let mut two = self.sent_messages.borrow_mut();
      
      one.push(String::from(message));
      two.push(String::from(message));
    }
  }

  let mock_messenger = MockMessenger::new();
  let mut limit_tracker = LimitTracker::new(&mock_messenger, 100);

  limit_tracker.set_value(80);
  assert_eq!(mock_messenger.sent_messages.borrow().len(), 1);
}

thread '<unnamed>' panicked at 'already borrowed: BorrowMutError', src/lib.rs:97:40
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::option::expect_none_failed
   3: <ctx::run_user_code_9::MockMessenger as ctx::Messenger>::send
   4: _run_user_code_9
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 101

In [24]:
#[derive(Debug)]
enum List {
  Cons(Rc<RefCell<i32>>, Rc<List>),
  Nil,
}

use List::{Cons, Nil};
use std::rc::Rc;
use std::cell::RefCell;

let value = Rc::new(RefCell::new(5));
let a = Rc::new(Cons(Rc::clone(&value), Rc::new(Nil)));
let b = Cons(Rc::new(RefCell::new(6)), Rc::clone(&a));
let c = Cons(Rc::new(RefCell::new(10)), Rc::clone(&a));

*(value.borrow_mut()) += 10;

println!("a after = {:?}", a);
println!("b after = {:?}", b);
println!("c after = {:?}", c);

a after = Cons(RefCell { value: 15 }, Nil)
b after = Cons(RefCell { value: 6 }, Cons(RefCell { value: 15 }, Nil))
c after = Cons(RefCell { value: 10 }, Cons(RefCell { value: 15 }, Nil))
